In [1]:
from agents.connection_finder import ConnectionFinder
from agents.explanation_builder import ExplanationBuilder

from IPython.display import display, HTML
import asyncio

In [2]:
connection_finder = ConnectionFinder()
explainer = ExplanationBuilder()

In [3]:
level = "beginner"
concept_a = "Quantum Entanglement"
concept_b = "Blockchain"
concept_a_knowledge = 0
concept_b_knowledge = 0
education_system = "french"
education_level = "high school"

profile = {
    "knowledge_level": level,
    "education_level": education_level,
    "education_system": education_system,
    "concept_a_knowledge": concept_a_knowledge,
    "concept_b_knowledge": concept_b_knowledge,
}

ctx = {
    "history": None,
    "level": level,
    "session_id": None,
    "concept_a": concept_a,
    "concept_b": concept_b,
    "profile": profile,
    "feedback_guidance": None
}

In [4]:
connections = await connection_finder.find(concept_a, concept_b, level, ctx)

In [5]:
path = connections["path"]

html = f"""
<style>
.path {{
    font-family: system-ui, -apple-system, BlinkMacSystemFont;
    font-size: 16px;
    padding: 8px 12px;
    background: #f8f9fa;
    border-radius: 8px;
    display: inline-block;
}}
.arrow {{
    color: #888;
    margin: 0 6px;
}}
.node {{
    font-weight: 600;
    color: #222;
}}
</style>

<div class="path">
    {"".join(
        f'<span class="node">{p}</span>' if i == len(path)-1
        else f'<span class="node">{p}</span><span class="arrow">→</span>'
        for i, p in enumerate(path)
    )}
</div>
"""

display(HTML(html))

#### **Note**: Sometimes, the json parsing can fail even though i've added a json extraction function. In case you get an error, or the cell outputs the results of the LLM as a json text instead of a json object, ``rerun the cell``, it should work

In [6]:
narrative = await explainer.build(
    connections,
    level,
    profile=profile,
    guidance=None,
    concept_a=concept_a,
    concept_b=concept_b,
)
            
narrative = narrative or {}
explanations = narrative.get("explanation")
analogies = narrative.get("analogies", [])

In [7]:
display(HTML(explanations))

In [8]:
for analogy in analogies:
    display(HTML(analogy))

# First Mitigation bias strategy: two new agents !!

In [9]:
from agents.bias_monitor import BiasMonitor
from agents.content_reviewer import ContentReviewer

bias = BiasMonitor()
reviewer = ContentReviewer()

In [10]:
bundle = {
    'connections': connections,
    'explanations': explanations,
    'analogies': analogies
}

In [11]:
bias_review_task = bias.review(bundle)
content_review_task = reviewer.evaluate(
    bundle,
    level=level,
    profile=profile,
    concept_a=concept_a,
    concept_b=concept_b,
)

bias_review, content_review = await asyncio.gather(bias_review_task, content_review_task)


In [12]:
biases = bias_review["raw"]
for bias_element in biases:
    display(HTML(bias_element))

In [13]:
content_review

{'level_alignment': True,
 'reading_level': 'High school / B1-B2 CEFR',
 'issues': [],
 'suggested_actions': [],
 'bias_risk': 'low'}

In [14]:
bias_review

{'has_bias': True,
 'raw': ["Cultural bias: The 'magic gloves' analogy is heavily reliant on a Western, magical-thinking understanding of the world. Suggest: Use a more universally relatable analogy, such as a synchronized dance or a carefully orchestrated event, to explain the instantaneous correlation.",
  "Language accessibility: The explanation uses dense terminology ('quantum entanglement,' 'distributed systems,' 'data integrity') without sufficient context or simplification. Suggest: Begin with a more accessible definition of quantum entanglement before introducing complex concepts, and provide simpler explanations for technical terms.",
  'Limited analogies: The analogies provided are somewhat simplistic and may not fully convey the depth of the concept. Suggest: Include a broader range of analogies, potentially drawing from fields like biology (e.g., a synchronized flock of birds) or even everyday social interactions (e.g., coordinated team effort) to aid understanding.',
  "De

In [15]:
mitigation_triggered = bias_review.get("has_bias") or not content_review.get("level_alignment", True)
mitigation_triggered

True

In [16]:
mitigation_triggered = bias_review.get("has_bias") or not content_review.get("level_alignment", True)

retry_strategies = [ 
    "CRITICAL: Address the following issues with high priority. ",
        "SIMPLIFY: Use simpler language and clearer structure. ",
            "RESTRUCTURE: Completely reorganize the explanation with a fresh approach. "
]

retry_count = 0
MAX_RETRIES = 2

new_narrative = narrative
new_explanations = explanations
new_analogies = analogies

while mitigation_triggered:
    if retry_count >= MAX_RETRIES:
        print("Max retries exceeded, no more bias mitigations attempts")
        break
    
    strategy = retry_strategies[retry_count]
    retry_count+=1
    suggestions = [strategy]

    if content_review and content_review.get("suggested_actions"):
        suggestions.append("Reviewer actions: " + ", ".join(content_review["suggested_actions"]))
    if bias_review and bias_review.get("raw"):
        suggestions.append("Bias adjustments: " + " | ".join(bias_review["raw"]))

    mitigation_guidance = " ".join(suggestions) if len(suggestions) !=0 else "Rewrite for clarity and inclusivity."
    new_narrative = await explainer.build(
        connections,
        level,
        profile=profile,
        guidance=mitigation_guidance,
        concept_a=concept_a,
        concept_b=concept_b,
    )
            
    new_narrative = new_narrative or {}
    new_explanations = new_narrative.get("explanation")
    new_analogies = new_narrative.get("analogies", [])

    new_bundle = {
        'connections': connections,
        'explanations': new_explanations,
        'analogies': new_analogies
    }

    # Re-review for bias and alignment
    new_bias_review_task = bias.review(new_bundle)
    new_content_review_task = reviewer.evaluate(
        new_bundle,
        level=level,
        profile=profile,
        concept_a=concept_a,
        concept_b=concept_b,
    )

    bias_review, content_review = await asyncio.gather(new_bias_review_task, new_content_review_task)
    mitigation_triggered = bias_review.get("has_bias") or not content_review.get("level_alignment", True)
    
    if not mitigation_triggered:
        break


Max retries exceeded, no more bias mitigations attempts


In [17]:
display(HTML(new_explanations))

In [18]:
for analogy in new_analogies:
    display(HTML(analogy))

# Second bias mitigation strategy: Adding user feedback after answer for better prompting

In [19]:
from agents.feedback_adapter import FeedbackAdapter

feedback = FeedbackAdapter()

feedback_rows = [
    {
        "comments": "use a more simple language, i don't understand the complex terms",
        "rating":1
    }
]

guidance = feedback.summarise(feedback_rows,level)

# We set the feedback guidance to None before, this will set it to our new guidance for the model
if ctx["feedback_guidance"] is None:
    ctx["feedback_guidance"] = guidance

This guidance text will serve as a base guidance for the model. If after that we need to mitigate the bias, it will be added to the mitigation bias strategy (cf *`_compose_guidance_with_strategy`* in **Orchestrator**)

In [20]:
narrative = await explainer.build(
    connections,
    level,
    profile=profile,
    guidance=guidance, # here we add the feedback guidance
    concept_a=concept_a,
    concept_b=concept_b,
)
            
narrative = narrative or {}
explanations = narrative.get("explanation")
analogies = narrative.get("analogies", [])

In [21]:
display(HTML(explanations))